### Modeling an Atom with Qiskit

https://medium.com/@avpol111/modeling-an-atom-with-qiskit-ab851b8ff6a5

In [2]:
# first, DistanceUnit for the distance between atoms in a molecule:
from qiskit_nature.units import DistanceUnit
# then, PySCFDriver to build our model (pyscf is a quantum chemistry library):
from qiskit_nature.second_q.drivers import PySCFDriver
# now, two solvers: the second one is a generic solver that will use the first
# one to actually solve the problem:
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
# finally, a mapper that maps spin operators onto fermionic creation and
# annihilation operators (it'll be used in the solver):
from qiskit_nature.second_q.mappers import JordanWignerMapper

https://qiskit.org/documentation/stable/0.24/tutorials/chemistry/01_electronic_structure.html

The Hamiltonian given in the previous section is expressed in terms of fermionic operators. To encode the problem into the state of a quantum computer, these operators must be mapped to spin operators (indeed the qubits follow spin statistics).

The Jordan-Wigner mapping is particularly interesting as it maps each Spin Orbital to a qubit (as shown on the Figure above).

Here we set up an object which contains all the information about any transformation of the fermionic Hamiltonian to the qubits Hamiltonian. In this example we simply ask for the Jordan-Wigner mapping.

https://qiskit-community.github.io/qiskit-nature/tutorials/06_qubit_mappers.html

The Jordan-Wigner mapping is the most straight-forward mapping with the simplest physical interpretation, because it maps the occupation of one spin-orbital to the occupation of one qubit.

https://qiskit-community.github.io/qiskit-nature/tutorials/03_ground_state_solvers.html#

After these steps, we need to define a solver. The solver is the algorithm through which the ground state is computed.

Let’s first start with a purely classical example: the NumPyMinimumEigensolver. This algorithm exactly diagonalizes the Hamiltonian. Although it scales badly, it can be used on small systems to check the results of the quantum algorithms.

To find the ground state we could also use the Variational Quantum Eigensolver (VQE) algorithm. The VQE algorithm works by exchanging information between a classical and a quantum computer.

In [3]:
# build the model:
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

# it'll return an instance of ElectronicStructureProblem:
problem = driver.run()

# and now, let's build a solver, get and print the results:
solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

result = solver.solve(problem)
print(result)

/home/tfg/TFG/tfg_env/lib/python3.10/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
                 (debye): [0.0  0.0  -0.000000003953]  Total: 0.000000003953
 


In [12]:
# build the model:
driver = PySCFDriver(
    atom="He 0 0 0",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

# it'll return an instance of ElectronicStructureProblem:
problem = driver.run()

# and now, let's build a solver, get and print the results:
solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

result = solver.solve(problem)
print(result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -2.80778395754
  - computed part:      -2.80778395754
~ Nuclear repulsion energy (Hartree): 0.0
> Total ground state energy (Hartree): -2.80778395754
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.0]
    - computed part:      [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


The nuclear repulsion energy is not included — that’s why we see zeros there

### VQE

https://qiskit-community.github.io/qiskit-nature/tutorials/03_ground_state_solvers.html#

https://docs.quantum.ibm.com/api/qiskit/primitives

https://qiskit-community.github.io/qiskit-algorithms/apidocs/qiskit_algorithms.optimizers.html

To find the ground state we could also use the Variational Quantum Eigensolver (VQE) algorithm. The VQE algorithm works by exchanging information between a classical and a quantum computer.

In [21]:
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

driver = PySCFDriver(
    atom="H 0 0 0",
    basis="sto3g",
    charge=0,
    spin=1,
    unit=DistanceUnit.ANGSTROM,
)

problem = driver.run()
mapper = JordanWignerMapper()



In [22]:
"""
In order to reduce the number of parameters and hence the size of the algorithmic
search space, an ansatz (German for guess) is often made regarding the form of the
ground state wavefunction, with this ansatz determining the exact configuration of
the parametrized quantum circuit. The choice of optimal ansatz varies across VQE
applications and is informed by the Hamiltonian of the system being studied.

The Hartree Fock initial state preparation circuit and the UCCSD variational unitary
transformation circuit (with HF as its initial state) were retrieved from a library of
optimized circuits in the qiskit_nature.second_q.circuit.library module.

Variational form: here we use the Unitary Coupled Cluster (UCC) ansatz (see for instance
[Physical Review A 98.2 (2018): 022322]). Since it is a chemistry standard, a factory is
already available allowing a fast initialization of a VQE with UCC. The default is to use
all single and double excitations. However, the excitation type (S, D, SD) as well as other
parameters can be selected. We also prepend the UCCSD variational form with a HartreeFock
initial state, which initializes the occupation of our qubits according to the problem which
we are trying solve.
"""

ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

"""
Estimator(): Estimator class estimates expectation values of quantum circuits and observables.

SLSQP(): Sequential Least SQuares Programming optimizer. This is the classical piece of code in
charge of optimizing the parameters in our variational form.
SLSQP minimizes a function of several variables with any combination of bounds, equality and
inequality constraints. The method wraps the SLSQP Optimization subroutine originally implemented
by Dieter Kraft.
"""

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

ValueError: The number of spatial orbitals 1must be greater than number of particles of any spin kind (1, 0).

In [ ]:
"""
One could also use any available ansatz / initial state or even define one’s own. For instance,

from qiskit_algorithms import VQE
from qiskit.circuit.library import TwoLocal

tl_circuit = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=2,
    parameter_prefix="y",
)

another_solver = VQE(Estimator(), tl_circuit, SLSQP())
"""

In [7]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030143
  - computed part:      -1.857275030143
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035694
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388949090541]
    - computed part:      [0.0  0.0  1.388949090541]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000390541]  Total: 0.000000390541
                 (debye): [0.0  0.0  -0.000000992656]  Total: 0.000000992656
 


### Excitations 

https://qiskit-community.github.io/qiskit-nature/stubs/qiskit_nature.second_q.circuit.library.UCC.html#

In [46]:
from qiskit_nature.second_q.circuit.library import UCC

mapper = JordanWignerMapper()

"""
Allowed characters are: 's' for singles, 'd' for doubles, 't' for triples, and 'q' for quadruples.
"""

ansatz = UCC(
    problem.num_spatial_orbitals,
    problem.num_particles,
    's',
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.836967991203
  - computed part:      -1.836967991203
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.116998996754
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
                 (debye): [0.0  0.0  -0.000000003953]  Total: 0.000000003953
 


### Excited states solvers

https://qiskit.org/documentation/stable/0.26/tutorials/chemistry/04_excited_states_solvers.html (No funciona)

https://qiskit-community.github.io/qiskit-nature/stubs/qiskit_nature.second_q.algorithms.ExcitedStatesEigensolver.html#

https://qiskit-community.github.io/qiskit-nature/tutorials/04_excited_states_solvers.html

In [44]:
from qiskit_nature.second_q.algorithms.excited_states_solvers import ExcitedStatesEigensolver
from qiskit_algorithms import NumPyEigensolver

numpy_solver = NumPyEigensolver(k=4, filter_criterion=problem.get_default_filter_criterion())
calc = ExcitedStatesEigensolver(mapper, numpy_solver)

res = calc.solve(problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -0.882722150245
> Total excited state energy (Hartree): -0.162753155796
  2: 
* Electronic excited state energy (Hartree): -0.224911252831
> Total excited state energy (Hartree): 0.495057741618
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  1:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  2:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
    